In [1]:
API_KEY = open("credentials.json", "r").read()

# Code to automatically add GPT-created tags to a Google Spreadsheet

## Install + Import Libraries

In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Imports for GPT Processing
import openai
import os
import pandas as pd
import time
from openai import OpenAI
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
#Imports for Google Sheets Implementation
import gspread
from oauth2client.service_account import ServiceAccountCredentials

C:\Users\hayden\AppData\Local\Temp\ipykernel_17604\2463684649.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## A list of functions for this code

In [4]:
########################################################
#This function asks GPT a question given string'prompt'#
########################################################
def ask_gpt(prompt):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=API_KEY,
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4-turbo",
        max_tokens = 300,
    )
    return chat_completion.choices[0].message.content
############################################################
#This function gets the raw text from a website given 'url'#
############################################################
def get_text_website(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    url = url
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text
#############################################################################
#This function converts a string of tags split by commas into a list of tags#
#############################################################################
def Convert(string):
    li = list(string.split(", "))
    return li
#############################################################################################################################
#This function removes all tags that were not in the provided list Asif and I put together (currently not in implementation)#
#############################################################################################################################
def remove_from_list(bigString, smallList):
    list_gpt_mock = smallList
    list_gpt_copy = list(list_gpt_mock)
    for substring in list_gpt_copy:
        if substring.lower() not in BIGSTRING.lower():
            list_gpt_mock.remove(substring)
    return list_gpt_mock
################################################################################
#This function converts the list of tags back into a string separated by commas#
################################################################################
def convert_list_to_gpt(smallList):
    tempString = ""
    for item in smallList:
        item = item.capitalize()
        tempString = tempString + item + ", "
    tempString = tempString[:-2]
    return tempString

## The following code is used to update every entry with GPT tags if they don't already have GPT tags

In [15]:
####################################################################################
#Import the Google Sheets Credentials and open up the IllinoisAdditions spreadsheet#
####################################################################################
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]
creds = ServiceAccountCredentials.from_json_keyfile_name("test-410522.json", scopes = scope)
file = gspread.authorize(creds)
workbook = file.open("IllinoisAdditions")
sheet = workbook.sheet1
#########################################################################################
#Check to see if there is a new entry (are there more form submissions than GPT tagged?)#
#########################################################################################
GPT_col = len(sheet.col_values(9))
form_col = len(sheet.col_values(1))
if GPT_col < form_col:
    for i in range(GPT_col+1, form_col+1):
        temp_url = sheet.cell(i,3)
        if temp_url.value is None:
            sheet.update_cell(i,9,"No website")
        else:
            try:
                temp_website_text = get_text_website(temp_url.value)
            except:
                sheet.update_cell(i,9,"Couldn't connect to website")
                continue
            temp_prompt = "Here is a list of tags separated by commas: American Indian Day, Holocaust and Genocide Study, Black History Study, Study of the History of Women, Irish Famine Study, Asian American History Study, Roles and Contributions of Ethnic Groups, African American, Polish, Lithuanian, German, Hungarian, Irish, Bohemian, Russian, Albanian, Italian, Czech, Slovak, French, Scots, Hispanics, Asian Americans, Deportation of Mexican-American Citizens, Roles and Contributions of LGBT People, Role of Labor Unions, Americans of Different Faith Practices, Muslim Americans, Jewish Americans, Christian Americans, Hindu Americans, Sikh Americans, Buddhist Americans, Disability History and Awareness, Study of the Process of Naturalization, Lief Erickson Day, Illinois Law Week, Historical Museum. FROM THE GIVEN LIST OF TAGS, give me a subset of tags (do not create your own tags!) that would apply to the given website, separated by commas (with no other text): " + temp_website_text
            if len(temp_prompt) > 10000:
                temp_prompt = temp_prompt[:10000]
            temp_gpt_output = ask_gpt(temp_prompt)
            if ":" in temp_gpt_output:
                colon_index = temp_gpt_output.find(":")
                temp_gpt_output = temp_gpt_output[(colon_index+2):]
            sheet.update_cell(i,9,temp_gpt_output)
    print("Finished updating tags!")
else:
    print("There were not tags to update.")

There were not tags to update.
